In [1]:
using Symbolics
using LinearAlgebra

In [45]:
function f_old(v)
    return [-v[2],v[1]]
end

function f(v)
    g = [-1 0; 0 1]
    return Symbolics.scalarize(g*v)
end

f (generic function with 1 method)

In [47]:
@variables vs[1:2], us[1:2]
g = [-1 0; 0 1]
g*vs

([-1 0; 0 1]*vs)[Base.OneTo(2)]

In [43]:

Symbolics.scalarize(g*vs)

2-element Vector{Num}:
 -vs[1]
  vs[2]

In [49]:
JFS_f = Symbolics.jacobian(f_old(vs),vs);
#JF_f_exp = Symbolics.build_function(JFS_f, f, vs);
#JFac_f = eval(JF_f_exp[1]);

LoadError: MethodError: no method matching similar(::Type{Matrix{Num}}, ::Tuple{Base.OneTo{Int64}, UnitRange{Int64}})
[0mClosest candidates are:
[0m  similar([91m::AbstractArray{T, N} where N[39m, ::Tuple) where T at abstractarray.jl:740
[0m  similar(::Type{T}, [91m::Union{Integer, AbstractUnitRange}...[39m) where T<:AbstractArray at abstractarray.jl:783
[0m  similar(::Type{A}, [91m::Union{Tuple{StaticArrays.SOneTo, Vararg{Union{Integer, Base.OneTo, StaticArrays.SOneTo}, N} where N}, Tuple{Union{Integer, Base.OneTo}, StaticArrays.SOneTo, Vararg{Union{Integer, Base.OneTo, StaticArrays.SOneTo}, N} where N}, Tuple{Union{Integer, Base.OneTo}, Union{Integer, Base.OneTo}, StaticArrays.SOneTo, Vararg{Union{Integer, Base.OneTo, StaticArrays.SOneTo}, N} where N}}[39m) where A<:AbstractArray at /Users/reula/.julia/packages/StaticArrays/OWJK7/src/abstractarray.jl:177
[0m  ...

In [55]:
function norm_g(v)
    g = [-1 0; 0 1]
    return Symbolics.scalarize(v'*g*v)
end
norm_g([1;2])

3

In [56]:
Grad_norm_g = Symbolics.gradient(norm_g(vs),vs);

LoadError: MethodError: no method matching similar(::Type{Vector{Num}}, ::Tuple{UnitRange{Int64}})
[0mClosest candidates are:
[0m  similar([91m::AbstractArray{T, N} where N[39m, ::Tuple) where T at abstractarray.jl:740
[0m  similar(::Type{T}, [91m::Union{Integer, AbstractUnitRange}...[39m) where T<:AbstractArray at abstractarray.jl:783
[0m  similar(::Type{A}, [91m::Union{Tuple{StaticArrays.SOneTo, Vararg{Union{Integer, Base.OneTo, StaticArrays.SOneTo}, N} where N}, Tuple{Union{Integer, Base.OneTo}, StaticArrays.SOneTo, Vararg{Union{Integer, Base.OneTo, StaticArrays.SOneTo}, N} where N}, Tuple{Union{Integer, Base.OneTo}, Union{Integer, Base.OneTo}, StaticArrays.SOneTo, Vararg{Union{Integer, Base.OneTo, StaticArrays.SOneTo}, N} where N}}[39m) where A<:AbstractArray at /Users/reula/.julia/packages/StaticArrays/OWJK7/src/abstractarray.jl:177
[0m  ...

In [97]:
function f(x)
    #return sum(x.*x)
    #return x[1]^2+x[2]^2
    return Symbolics.scalarize(sum(x.*x))
end

@variables v[1:2]

1-element Vector{Symbolics.Arr{Num, 1}}:
 v[1:2]

In [98]:
f(v)

v[1]^2 + v[2]^2

In [106]:
#∇f = Symbolics.gradient(f(v),[v[1],v[2]]);
s = [v[i] for i ∈ 1:2]
∇f = Symbolics.gradient(f(v),s);

In [107]:
∇f

2-element Vector{Num}:
 2v[1]
 2v[2]

In [130]:
#g = [-1. 0.; 0. 1.]

@variables l[1:2,1:2] 

function h(x,g)   
    #xs = Symbolics.scalarize(x)
    #return xs'*g*xs
    Symbolics.scalarize(x)'*Symbolics.scalarize(g*x)
end
#h(v,l)

Symbolics.scalarize(v)'*Symbolics.scalarize(l*v)

(v[1]*l[1, 1] + v[2]*l[1, 2])*v[1] + (v[1]*l[2, 1] + v[2]*l[2, 2])*v[2]

In [132]:
∇h = Symbolics.gradient(h(v,l),s;simplify=true)

2-element Vector{Num}:
 (l[1, 2] + l[2, 1])*v[2] + 2v[1]*l[1, 1]
 (l[1, 2] + l[2, 1])*v[1] + 2v[2]*l[2, 2]

In [138]:
∇h_exp = Symbolics.build_function(∇h,v,l);
∇h_e = eval(∇h_exp[1]);
∇h_e([1.,1],[-1.0 0.0;0.0 1.0])

2-element Vector{Float64}:
 -2.0
  2.0

In [140]:
function st(t)
    u = ones(2)
    u[1] = -1.0
    g = Diagonal(u) + zeros(2,2)
    return τ₂= tr(Symbolics.scalarize(t)*g'*Symbolics.scalarize(t)*g)
end

st (generic function with 1 method)

In [141]:
st([1. 0.; 0. 1.])

2.0

In [144]:
@variables ts[1:2,1:2]

1-element Vector{Symbolics.Arr{Num, 2}}:
 ts[1:2,1:2]

In [174]:
tst = vec([ts[i,j] for i ∈ 1:2, j ∈ 1:2])
∇st = Symbolics.gradient(st(ts),tst;simplify=true)

4-element Vector{Num}:
 2ts[1, 1]
        -2.0ts[1, 2]
        -2.0ts[2, 1]
 2ts[2, 2]

In [173]:
r = [1.; 2; 3; 4]
reshape(∇st, 2,2)'

2×2 adjoint(::Matrix{Num}) with eltype Num:
 2ts[1, 1]                   -2.0ts[1, 2]
        -2.0ts[2, 1]  2ts[2, 2]

In [169]:
hvcat((2,2), Tuple(r[i] for i ∈ 1:4))

LoadError: BoundsError: attempt to access Tuple{NTuple{4, Float64}} at index [2]

In [170]:
hvcat((2,2), (1, 2, 3, 4))

LoadError: BoundsError: attempt to access Tuple{NTuple{4, Int64}} at index [2]